In [ ]:
import torch
import sys
import os
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy
from datetime import datetime
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
	sys.path.insert(0, module_path)
print(sys.path)

import Double_Pendulum.Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics
import Plotting.pendulum_plot as pendulum_plot


import plot_collocated as plt_col

from functools import partial

import matplotlib
matplotlib.rcParams['font.family']   = 'serif'
matplotlib.rcParams['font.serif']    = ['Times New Roman']
matplotlib.rcParams['mathtext.fontset'] = 'dejavuserif'



%load_ext autoreload
%autoreload 2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
rp = robot_parameters.LUMPED_PARAMETERS
plotter = pendulum_plot.Anim_plotter(rp)


model_cw = False



model_ana = autoencoders.Analytic_transformer(rp)

In [ ]:
# Controller gains in theta-space
Kp = torch.tensor([[100., 0.], 
				   [0., 0.]]).to(device)
Kd = torch.tensor([[40., 0.], 
				   [0., 0.]]).to(device)

# Controller gains in q-space
Kp_naive = torch.tensor([[100., 100.]]).to(device)
Kd_naive = torch.tensor([[40., 40.]]).to(device)

K_naive = torch.cat((Kp_naive, Kd_naive), dim=1).squeeze(0)

#C_damp = torch.tensor([[12., 0.], [0., 12.]]).to(device)

In [ ]:
settling_time_threshold = 0.05

dt = 0.01
t_end = 0.3
t_series = torch.arange(0, t_end, dt)

#### Define desired conditions

In [ ]:
def desired_conditions(xy_des, model):
	q_des = transforms.inverse_kinematics(xy_des, rp, is_clockwise=model_cw).unsqueeze(0)
	q_d_des = torch.tensor([[0., 0.]]).requires_grad_().to(device)

	is_clockwise_des = transforms.check_clockwise(q_des.squeeze(0))

	th_des_ana = model_ana.encoder_vmap(q_des)
	th_d_des_ana = (model_ana.jacobian_enc(q_des) @ q_d_des.T).T

	th_des_est = model.encoder_vmap(q_des)
	th_d_des_est = (model.jacobian_enc(q_des) @ q_d_des.T).T

	q_des_est = model.decoder_vmap(th_des_est, is_clockwise_des)
	q_d_des_est = (model.jacobian_dec(th_des_est, clockwise=is_clockwise_des) @ th_d_des_est.T).T
	xy_des_est, _ = transforms.forward_kinematics(rp, q_des_est[0])

	#print("xy_des:", xy_des.detach().cpu().numpy())
	#print("th_des_ana:", th_des_ana.detach().cpu().numpy()[0])

	return (q_des, q_d_des, is_clockwise_des, th_des_ana, th_d_des_ana, 
		 	th_des_est, th_d_des_est, q_des_est, q_d_des_est, xy_des_est)

#### Define start conditions

In [ ]:
def start_conditions(xy_start, model):
	q_start = transforms.inverse_kinematics(xy_start, rp, is_clockwise=model_cw).unsqueeze(0)
	q_d_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)
	q_dd_start = torch.tensor([[0., 0.]]).requires_grad_().to(device)

	is_clockwise_start = transforms.check_clockwise(q_start.squeeze(0))

	th_start_est = model.encoder_vmap(q_start)
	th_d_start_est = (model.jacobian_enc(q_start) @ q_d_start.T).T

	th_start_ana = model_ana.encoder_vmap(q_start)
	th_d_start_ana = (model_ana.jacobian_enc(q_start) @ q_d_start.T).T

	q_start_est = model.decoder_vmap(th_start_est, is_clockwise_start)
	q_d_start_est = (model.jacobian_dec(th_start_est, clockwise=is_clockwise_start) @ th_d_start_est.T).T
	xy_start_est, _ = transforms.forward_kinematics(rp, q_start_est[0])

	#print("xy_start:", xy_start.detach().cpu().numpy())
	#print("th_start_ana:", th_start_ana.detach().cpu().numpy()[0])

	return (q_start, q_d_start, q_dd_start, is_clockwise_start, th_start_est, th_d_start_est, 
		 	th_start_ana, th_d_start_ana, q_start_est, q_d_start_est, xy_start_est)

#### NN simulation

In [ ]:
def sim_collocated_nn(xy_start, xy_des, timestamp):

	(q_des, q_d_des, is_clockwise_des, th_des_ana, th_d_des_ana, 
		th_des_est, th_d_des_est, q_des_est, q_d_des_est, xy_des_est) = desired_conditions(xy_des, model_nn)

	(q_start, q_d_start, q_dd_start, is_clockwise_start, th_start_est, th_d_start_est, 
		th_start_ana, th_d_start_ana, q_start_est, q_d_start_est, xy_start_est) = start_conditions(xy_start, model_nn)
	

	num_steps = int(t_end / dt)
	th_series, th_d_series      = torch.empty((num_steps, 2)).to(device), torch.empty((num_steps, 2)).to(device)
	th_ana_series, th_d_ana_series = torch.empty((num_steps, 2)).to(device), torch.empty((num_steps, 2)).to(device)
	q_hat_series, q_d_hat_series = torch.empty((num_steps, 2)).to(device), torch.empty((num_steps, 2)).to(device)
	q_real_series, q_d_real_series, q_dd_real_series = torch.empty((num_steps, 2)).to(device), torch.empty((num_steps, 2)).to(device), torch.empty((num_steps, 2)).to(device)
	u_series = torch.empty((num_steps, 1)).to(device)
	tau_q_series = torch.empty((num_steps, 2)).to(device)

	# Define starting conditions
	th = th_start_est
	th_d = th_d_start_est

	q_hat = q_start_est
	q_d_hat = q_d_start_est

	q_real, q_d_real, q_dd_real = q_start, q_d_start, q_dd_start

	is_clockwise = transforms.check_clockwise(q_start.squeeze(0))


	


	for i, t in enumerate(torch.arange(0, t_end, dt)):
		t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
		if torch.allclose(t, torch.floor(t)) and torch.floor(t)%5 == 0:
			print(t_string)

		is_clockwise = transforms.check_clockwise(q_real.squeeze(0))
		if model_cw != is_clockwise:
			print("fixing value to", "clockwise" if model_cw else "counterclockwise")
			q = transforms.flip_q(rp, q_real.squeeze(0), model_cw).unsqueeze(0)
			q_d = transforms.flip_q_d(rp, q_real.squeeze(0), q_d_real, model_cw)
		else:
			q = q_real
			q_d = q_d_real

		th = model_nn.encoder_vmap(q)
		th_d = (model_nn.jacobian_enc(q) @ q_d.T).T
		
		q_hat = model_nn.decoder_vmap(th, clockwise=model_cw)
		q_d_hat = (model_nn.jacobian_dec(th) @ th_d.T).T

		is_clockwise = transforms.check_clockwise(q_hat.squeeze(0))

		""" Obtain Jacobian, dynamical matrices"""
		
		J_h_inv = model_nn.jacobian_dec(th, is_clockwise).squeeze(0)
		J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

		M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
		A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


		""" Feed-forward simulation of the system, not on real dynamics """

		M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
		A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans)

		u = torch.pinverse(A_th) @ ((Kp @ (th_des_est - th).T + Kd @ (th_d_des_est - th_d).T))
		u = u + torch.pinverse(A_th) @ G_th

		""" Update the real system and apply latent control input. """

		M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_real.squeeze(0), q_d_real.squeeze(0))
		A_q_real = dynamics.input_matrix(rp, q_real.squeeze(0))

		tau_q_real = A_q_real * u
		q_dd_real = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_real).T)- G_q_real)).T
		q_d_real = q_d_real + q_dd_real * dt
		q_real = q_real + q_d_real * dt
		q_real = transforms.wrap_to_pi(q_real)
		
		th = model_nn.encoder_vmap(q_real)
		q_hat = model_nn.decoder_vmap(th, clockwise=transforms.check_clockwise(q_real.squeeze(0)))
		q_hat = transforms.wrap_to_pi(q_hat)
		th_d = (model_nn.jacobian_enc(q_real) @ q_d_real.T).T
		q_d_hat = (model_nn.jacobian_dec(th, clockwise=is_clockwise) @ th_d.T).T

		th_ana = model_ana.encoder_vmap(q_real)
		th_d_ana = (model_ana.jacobian_enc(q_real) @ q_d_real.T).T



		""" Store data for plotting """

		th_series[i] = th.detach()
		th_d_series[i] = th_d.detach()

		th_ana_series[i] = th_ana.detach()
		th_d_ana_series[i] = th_d_ana.detach()

		q_real_series[i] = q_real.detach()
		q_d_real_series[i] = q_d_real.detach()
		q_dd_real_series[i] = q_dd_real.detach()

		q_hat_series[i] = q_hat.detach()
		q_d_hat_series[i] = q_d_hat.detach()

		u_series[i] = u.detach()
		tau_q_series[i] = tau_q_real.T.detach()


	xy_real_series = torch.empty(q_real_series.size()).to(device)
	xy_hat_series = torch.empty(q_real_series.size()).to(device)

	for i, (q_real, q_hat) in enumerate(zip(q_real_series, q_hat_series)):
		xy_real, _ = transforms.forward_kinematics(rp, q_real)
		xy_real_series[i] = xy_real.unsqueeze(0)
		xy_hat, _ = transforms.forward_kinematics(rp, q_hat)
		xy_hat_series[i] = xy_hat.unsqueeze(0)

	nn_sim_data = {
		"th_s": th_series,
		"th_d_s": th_d_series,
		"th_ana_s": th_ana_series,
		"th_d_ana_s": th_d_ana_series,
		"q_real_s": q_real_series,
		"q_d_real_s": q_d_real_series,
		"q_dd_real_s": q_dd_real_series,
		"q_hat_s": q_hat_series,
		"q_d_hat_s": q_d_hat_series,
		"xy_real_s": xy_real_series,
		"xy_hat_s": xy_hat_series,
		"u_s": u_series,
		"tau_q_s": tau_q_series
	}

	return (nn_sim_data, q_des, th_des_ana, th_start_ana)

#### Analytic simulation

In [ ]:
def sim_collocated_ana(xy_start, xy_des, timestamp, model):


	(q_des, q_d_des, is_clockwise_des, th_des_ana, th_d_des_ana, 
		th_des_est, th_d_des_est, q_des_est, q_d_des_est, xy_des_est) = desired_conditions(xy_des, model_ana)

	(q_start, q_d_start, q_dd_start, is_clockwise_start, th_start_est, th_d_start_est, 
		th_start_ana, th_d_start_ana, q_start_est, q_d_start_est, xy_start_est) = start_conditions(xy_start, model_ana)

	th_ana_series, th_d_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
	q_ana_series, q_d_ana_series, q_dd_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
	q_hat_ana_series, q_d_hat_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	u_ana_series = torch.empty((0,1)).to(device)
	tau_q_ana_series = torch.empty((0,2)).to(device)

	# Define starting angles
	q, q_d, q_dd = q_start, q_d_start, q_dd_start

	is_clockwise = False

	

	for t in torch.arange(0, t_end, dt):
		t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
		if torch.allclose(t, torch.floor(t)) and torch.floor(t)%5 == 0:
			print(t_string)


		th = model.encoder_vmap(q)
		th_d = (model.jacobian_enc(q) @ q_d.T).T
		
		q_hat = model.decoder_vmap(th, clockwise=model_cw)
		q_d_hat = (model.jacobian_dec(th) @ th_d.T).T

		""" Obtain Jacobian, dynamical matrices"""
		
		J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
		J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

		M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
		A_q_est = dynamics.input_matrix(rp, q_hat.squeeze(0))


		""" Feed-forward simulation of the system, not on real dynamics """

		M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th, th_d, J_h_inv, J_h_inv_trans)
		A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans)

		u = torch.pinverse(A_th) @ ((Kp @ (th_des_ana - th).T + Kd @ (th_d_des_ana - th_d).T))
		u = u + torch.pinverse(A_th) @ G_th

		""" Update the real system and apply latent control input. """

		M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q_real = dynamics.input_matrix(rp, q.squeeze(0))

		tau_q_real = A_q_real * u
		q_dd = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d).T)- G_q_real)).T  #  - C_damp @ ((q_d_real).T) 
		q_d = q_d + q_dd * dt
		q = q + q_d * dt
		q = transforms.wrap_to_pi(q)
		
		th_ana = model.encoder_vmap(q)
		th_d_ana = (model.jacobian_enc(q) @ q_d.T).T


		""" Store data for plotting """

		th_ana_series = torch.cat((th_ana_series, th_ana.detach()), dim=0)
		th_d_ana_series = torch.cat((th_d_ana_series, th_d_ana.detach()), dim=0)
		
		q_ana_series = torch.cat((q_ana_series, q.detach()), dim=0)
		q_d_ana_series = torch.cat((q_d_ana_series, q_d.detach()), dim=0)
		q_dd_ana_series = torch.cat((q_dd_ana_series, q_dd.detach()), dim=0)

		q_hat_ana_series = torch.cat((q_hat_ana_series, q_hat.detach()), dim=0)
		q_d_hat_ana_series = torch.cat((q_d_hat_ana_series, q_d_hat.detach()), dim=0)

		u_ana_series = torch.cat((u_ana_series, u.detach()), dim=0)
		tau_q_ana_series = torch.cat((tau_q_ana_series, tau_q_real.T.detach()), dim=0)

		print("")



	xy_real_series = torch.empty(q_ana_series.size()).to(device)
	xy_hat_series = torch.empty(q_hat_ana_series.size()).to(device)

	for i, (q_real, q_hat) in enumerate(zip(q_ana_series, q_hat_ana_series)):
		xy_real, _ = transforms.forward_kinematics(rp, q_real)
		xy_real_series[i] = xy_real.unsqueeze(0)
		xy_hat, _ = transforms.forward_kinematics(rp, q_hat)
		xy_hat_series[i] = xy_hat.unsqueeze(0)

	ana_sim_data = {
		"th_s": th_ana_series,
		"th_d_s": th_d_ana_series,
		"q_ana_s": q_ana_series,
		"q_d_ana_s": q_d_ana_series,
		"q_dd_ana_s": q_dd_ana_series,
		"q_hat_ana_s": q_hat_ana_series,
		"q_d_hat_ana_s": q_d_hat_ana_series,
		"u_ana_s": u_ana_series,
		"tau_q_ana_s": tau_q_ana_series,
		"xy_real_s": xy_real_series,
		"xy_hat_s": xy_hat_series,
	}

	return ana_sim_data


		


#### Naive simulation (BEING WRITTEN)

In [ ]:
def sim_collocated_naive(xy_start, xy_des, timestamp):


	(q_des, q_d_des, is_clockwise_des, th_des_ana, th_d_des_ana, 
		th_des_est, th_d_des_est, q_des_est, q_d_des_est, xy_des_est) = desired_conditions(xy_des, model_ana)

	(q_start, q_d_start, q_dd_start, is_clockwise_start, th_start_est, th_d_start_est, 
		th_start_ana, th_d_start_ana, q_start_est, q_d_start_est, xy_start_est) = start_conditions(xy_start, model_ana)

	th_naive_series, th_d_naive_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
	q_naive_series, q_d_naive_series, q_dd_naive_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)

	naive_des = torch.cat((q_des.T, q_d_des.T), dim = 0)

	

	u_naive_series = torch.empty((0,1)).to(device)
	tau_q_naive_series = torch.empty((0,2)).to(device)

	q, q_d, q_dd = q_start, q_d_start, q_dd_start

	is_clockwise = False

	model = autoencoders.Analytic_transformer(rp)

	for t in torch.arange(0, t_end, dt):
		t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
		if torch.allclose(t, torch.floor(t)) and torch.floor(t)%5 == 0:
			print(t_string)


		""" Obtain Jacobian, dynamical matrices"""
		M_q_naive, C_q_naive, G_q_naive = dynamics.dynamical_matrices(rp, q.squeeze(0), q_d.squeeze(0))
		A_q_naive = dynamics.input_matrix(rp, q.squeeze(0))


		""" Feed-forward simulation of the system, not on real dynamics """

		naive_state = torch.cat((q.T, q_d.T), dim = 0)
		print("naive state:", naive_state)

		naive_error = naive_state - naive_des
		delta_u = K_naive @ naive_error
		print("delta_u:", delta_u)
		u = delta_u + torch.pinverse(A_q_naive) @ G_q_naive

		""" Update the real system and apply latent control input. """


		tau_q_naive = A_q_naive * u
		q_dd = (torch.pinverse(M_q_naive) @ (tau_q_naive - C_q_naive @ ((q_d).T)- G_q_naive)).T  #  - C_damp @ ((q_d_real).T) 
		q_d = q_d + q_dd * dt
		q = q + q_d * dt
		q = transforms.wrap_to_pi(q)
		
		th_naive = model.encoder_vmap(q)
		th_d_naive = (model.jacobian_enc(q) @ q_d.T).T


		""" Store data for plotting """

		th_naive_series = torch.cat((th_naive_series, th_naive.detach()), dim=0)
		th_d_naive_series = torch.cat((th_d_naive_series, th_d_naive.detach()), dim=0)
		
		q_naive_series = torch.cat((q_naive_series, q.detach()), dim=0)
		q_d_naive_series = torch.cat((q_d_naive_series, q_d.detach()), dim=0)
		q_dd_naive_series = torch.cat((q_dd_naive_series, q_dd.detach()), dim=0)

		u_naive_series = torch.cat((u_naive_series, u.detach()), dim=0)
		tau_q_naive_series = torch.cat((tau_q_naive_series, tau_q_naive.T.detach()), dim=0)

		print("")


	xy_naive_series = torch.empty(q_naive_series.size()).to(device)

	for i, q_naive in enumerate(q_naive_series):
		xy_naive, _ = transforms.forward_kinematics(rp, q_naive)
		xy_naive_series[i] = xy_naive.unsqueeze(0)

	naive_sim_data = {
		"th_naive_s": th_naive_series,
		"th_d_naive_s": th_d_naive_series,
		"q_naive_s": q_naive_series,
		"q_d_naive_s": q_d_naive_series,
		"q_dd_naive_s": q_dd_naive_series,
		"u_naive_s": u_naive_series,
		"tau_q_naive_s": tau_q_naive_series,
		"xy_naive_s": xy_naive_series,
	}

	return naive_sim_data
		

In [ ]:
def calc_settling_time(th_0_abs_error_series, th_des_ana, th_start_ana, settling_time_threshold = 0.05):

	"""
	Function to calculate settling time of a simulation, or return  
	"""

	th_distance = torch.abs(th_des_ana[0,0] - th_start_ana[0,0])
	tolerance = settling_time_threshold * th_distance
	print("Tolerance distance from th_des:", round(tolerance.item(), 5), "[m]")

	within_tolerance = th_0_abs_error_series <= tolerance
	match_indices = torch.nonzero(within_tolerance).squeeze()
	print("THIS SHOULD PROBABLY TAKE THE LAST MATCH")

	if match_indices.numel() > 0:
		first_time_idx = match_indices[0].item()
		settling_time = first_time_idx * dt
		print(f"Reached within 5% tolerance at t = {settling_time:.2f} seconds.")
	else:
		print("Did not reach within 5% tolerance.")
		settling_time = torch.inf

	return settling_time, tolerance, match_indices

def calc_rmse(theta_real_series, th_des_ana, match_indices):


	if match_indices.numel() > 0:
		first_time_idx = match_indices[0].item()
		settled_theta = theta_real_series[3*first_time_idx:, 0]
		rmse = torch.sqrt(torch.mean((settled_theta - th_des_ana[0, 0]) ** 2))
		print(f"RMSE after 3x settling time: {rmse.item():.6f} [m]")
	else:
		print("Did not reach within 5% tolerance.")
		settling_time = torch.inf
		rmse = torch.inf 

	return rmse
    

In [ ]:
def plot_settling(th_0_error_series, tolerance, settling_time):
	# Time vector (same length as error series)
	t_series = torch.arange(0, t_end, dt)

	# Detach if needed (for plotting outside torch computation graph)
	error_np = th_0_error_series.detach().cpu().numpy()
	t_np = t_series.cpu().numpy()

	# Plot
	plt.figure(figsize=(4, 3))
	plt.plot(t_np, error_np, label=r"$\bar{θ}_{0} - θ_{0}$", color='tab:blue')
	plt.axhline(y=-tolerance.item(), color='tab:red', linestyle='--', label="-5% Tolerance")
	plt.axhline(y=tolerance.item(), color='tab:red', linestyle='--', label="5% Tolerance")
	plt.axvline(x=settling_time, color='tab:green', linestyle='--', label="Settling Time")

	plt.title("Error of " + r"$θ_0$" + " vs Time")
	plt.xlabel("Time [s]")
	plt.ylabel("Error [m]")
	plt.xlim(0, t_end)
	plt.legend()
	plt.grid(True)
	plt.tight_layout()
	plt.show()

#### Trial data generation

In [ ]:
n_sims = 4

x_des_min, x_des_max = rp["xa"], rp["xa"]
y_des_min, y_des_max = -3., 1.
x_des_series = torch.rand(n_sims) * (x_des_max - x_des_min) + x_des_min
y_des_series = torch.rand(n_sims) * (y_des_max - y_des_min) + y_des_min
xy_des_series = torch.stack((x_des_series, y_des_series), dim=1).requires_grad_().to(device)

x_start_min, x_start_max = 1., 3.
y_start_min, y_start_max = -3., 1.
x_start_series = torch.rand(n_sims) * (x_start_max - x_start_min) + x_start_min
y_start_series = torch.rand(n_sims) * (y_start_max - y_start_min) + y_start_min
xy_start_series = torch.stack((x_start_series, y_start_series), dim=1).requires_grad_().to(device)

print(xy_start_series)
print(xy_des_series)

diff = xy_des_series - xy_start_series
print("\n", diff)

#### Analytic simulation

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

th_ana_series, th_d_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
q_ana_series, q_d_ana_series, q_dd_ana_series = torch.empty((0,2)).to(device), torch.empty((0,2)).to(device), torch.empty((0,2)).to(device)
u_ana_series = torch.empty((0,1)).to(device)
tau_q_ana_series = torch.empty((0,2)).to(device)


dt = 0.01
t_end = 15
t_series = torch.arange(0, t_end, dt)

th = th_start_ana
th_d = th_d_start_ana


q_ana, q_d_ana, q_dd_ana = q_start, q_d_start, q_dd_start

is_clockwise = False

model = autoencoders.Analytic_transformer(rp)

for t in torch.arange(0, t_end, dt):
	t_string = "Time: [" + str(t.item().__round__(3)) + "/" + str(t_end) + ".0]"
	print(t_string)


	q_ana
	q_d_ana

	th_ana = model.encoder_vmap(q_ana)
	th_d_ana = (model.jacobian_enc(q_ana) @ q_d_ana.T).T
	
	q_hat_ana = model.decoder_vmap(th_ana, clockwise=model_cw)
	q_d_hat_ana = (model.jacobian_dec(th_ana) @ th_d_ana.T).T

	""" Obtain Jacobian, dynamical matrices"""
	
	J_h_inv = model.jacobian_dec(th, is_clockwise).squeeze(0)
	J_h_inv_trans = torch.transpose(J_h_inv, 0, 1)

	M_q_est, C_q_est, G_q_est = dynamics.dynamical_matrices(rp, q_hat_ana.squeeze(0), q_d_hat_ana.squeeze(0))
	A_q_est = dynamics.input_matrix(rp, q_hat_ana.squeeze(0))


	""" Feed-forward simulation of the system, not on real dynamics """

	M_th, C_th, G_th = transforms.transform_dynamical_from_inverse(M_q_est, C_q_est, G_q_est, th_ana, th_d_ana, J_h_inv, J_h_inv_trans)
	A_th = transforms.transform_input_matrix_from_inverse_trans(A_q_est, J_h_inv_trans)

	u = torch.pinverse(A_th) @ ((Kp @ (th_des - th_ana).T + Kd @ (th_d_des - th_d_ana).T))
	u = u + torch.pinverse(A_th) @ G_th

	""" Update the real system and apply latent control input. """

	M_q_real, C_q_real, G_q_real = dynamics.dynamical_matrices(rp, q_ana.squeeze(0), q_d_ana.squeeze(0))
	A_q_real = dynamics.input_matrix(rp, q_ana.squeeze(0))

	tau_q_real = A_q_real * u
	q_dd_ana = (torch.pinverse(M_q_real) @ (tau_q_real - C_q_real @ ((q_d_ana).T)- G_q_real)).T  #  - C_damp @ ((q_d_real).T) 
	q_d_ana = q_d_ana + q_dd_ana * dt
	q_ana = q_ana + q_d_ana * dt
	q_ana = transforms.wrap_to_pi(q_ana)
	
	th_ana = model.encoder_vmap(q_ana)
	th_d_ana = (model.jacobian_enc(q_ana) @ q_d_ana.T).T


	""" Store data for plotting """

	th_ana_series = torch.cat((th_ana_series, th_ana.detach()), dim=0)
	th_d_ana_series = torch.cat((th_d_ana_series, th_d_ana.detach()), dim=0)
	
	q_ana_series = torch.cat((q_ana_series, q_ana.detach()), dim=0)
	q_d_ana_series = torch.cat((q_d_ana_series, q_d_ana.detach()), dim=0)
	q_dd_ana_series = torch.cat((q_dd_ana_series, q_dd_ana.detach()), dim=0)

	u_ana_series = torch.cat((u_ana_series, u.detach()), dim=0)
	tau_q_ana_series = torch.cat((tau_q_ana_series, tau_q_real.T.detach()), dim=0)

	print("")

	
	
